# Indian Pines

In [ ]:
import scipy.io as sio
import sklearn.metrics

from tensorflow import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

from Source_Model_Utils import *

from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
uIndian_pines = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Indian Pines\\Indian_pines_corrected.mat')
gt_uIndian_pines = sio.loadmat('C:\\Users\\de991521\\Desktop\\EE_297_PROJECT\\Indian Pines\\Indian_pines_gt.mat')
data_IN = uIndian_pines['indian_pines_corrected']
gt_IN = gt_uIndian_pines['indian_pines_gt']
print(data_IN.shape, gt_IN.shape)
data_IN = np.moveaxis(data_IN, 2, 0)
print(data_IN.shape, gt_IN.shape)

In [ ]:
values,counts = np.unique(gt_IN, return_counts=True)
print(values,counts)
range_of_class = list(values)
if 0 in range_of_class:
    range_of_class.pop(0)
range_of_class

In [ ]:
overlap_ratio = 0.90
Xtrain, Xtest, Ytrain, Ytest, class_len, counts = prepare_data_for_training(range_of_class = range_of_class, Cube_size = 10, Data = data_IN , Gt = gt_IN, small_segmented_1 = [], small_seg_gt_1 = [], percentage = 80, overlap_ratio=overlap_ratio, ch=103)
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

In [ ]:
model_1 = model(input_shape = Xtrain[0].shape, classes = len(range_of_class))
model_1.summary()

In [ ]:
model_checkpoint = ModelCheckpoint('Trained Models\\model_full_best_pavia_overlap_ratio_'+ str(int(overlap_ratio*100)) + '_percent.h5', monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
model_1.compile(optimizer= keras.optimizers.SGD(lr=0.0001, decay=1e-5, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_1.fit(Xtrain, Ytrain, epochs = 500, batch_size = 16, validation_data=(Xtest,Ytest), verbose=1, callbacks=[model_checkpoint])

In [ ]:
model_1.layers[0:35]

In [ ]:
model_new = Model(input=model_1.layers[0].input, output=model_1.layers[35].output)
model_new.save("model_indian.h5")
model_new.summary()

In [ ]:
preds = model_1.evaluate(Xtest, Ytest)
# print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
y_pred = model_1.predict(Xtest, verbose=1)

In [ ]:
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(Ytest, axis=1), np.argmax(y_pred, axis=1))
confusion_matrix

In [ ]:
counts